In [2]:
!ls -l
import json

total 256
-rw-r--r--  1 yoavfreund  staff  10349 Sep 17 21:35 Function pointers and multiprocess communication.ipynb
-rw-r--r--  1 yoavfreund  staff   2878 Sep 17 12:57 Problem2.3.json
-rw-r--r--  1 yoavfreund  staff  65469 Sep 17 12:57 Problem8Attempts.json
-rw-r--r--  1 yoavfreund  staff   8988 Sep 17 12:57 Process Data.ipynb
-rw-r--r--  1 yoavfreund  staff  36459 Sep 17 12:57 attempts.txt


In [3]:
def answer_should_be_int(att_eval_tree):
    if len(att_eval_tree)==1:
        final_value=att_eval_tree[0]
    elif len(att_eval_tree)>1:
        final_value=att_eval_tree[0][1]
    else:
        final_value=0
    if int(final_value)!=final_value:
        return "Can the answer to this question be a fractional number?(y,n)  [__]{'n'}"
    else:
        return ""
        
def answer_is_not_expression(att_eval_tree):
    if len(att_eval_tree)==1:
        return "Please enter an expression, not just final numerical result"
    else:
        return ""

In [4]:
def check_81(params):
    attempt,att_parse_tree,att_eval_tree,d1,d2,d3,variables = params

    hint=answer_should_be_int(att_eval_tree)
    if  len(hint)!=0: 
        return hint
    
    hint=answer_is_not_expression(att_eval_tree)
    if  len(hint)!=0: 
        return hint

    return None

In [40]:
def flatten(L):
    #print 'flattening ',L,type(L),
    #if type(L)==list:
        #print 'len(L)=',len(L),'type(L[0])=',type(L[0]),
    if type(L)!=list:
        #print 'not list'
        return None
    elif len(L)==1 and type(L[0])==float:
        #print 'list of one number'
        return [{'val':L[0]}]
    elif len(L)==3 and type(L[0])==unicode:
        #print 'list of one clause'
        return [{'val':L[1],'location':L[2]}]
    else:
        #print 'general list'
        answer=[]
        for item in L:
            answer = answer+flatten(item)
        return answer
        
        
        

In [42]:
file=open('Problem8Attempts.json','r');
answer=json.loads(file.readline())['answer']
print 'answer=',answer
for line in file.readlines():
    params=json.loads(line);
    attempt,att_parse_tree,att_eval_tree,d1,d2,d3,variables = params
    hint=check_81(params)
    
    #if hint !=None:
    #    print attempt,hint
    
    if hint==None:
        print attempt,answer,att_eval_tree,variables
        flat=flatten(att_eval_tree)
        print 'flattened=',flat
        print 'set of values=',set([x['val'] for x in flat if x.has_key('val')])


answer= 26**$n2-25**$n2
6^2-5^2 26**$n2-25**$n2 [[u'-', 11.0, [0, 6]], [[u'^', 36.0, [0, 2]], [6.0], [2.0]], [[u'^', 25.0, [4, 6]], [5.0], [2.0]]] {u'$n2': 5.0}
flattened= [{'location': [0, 6], 'val': 11.0}, {'location': [0, 2], 'val': 36.0}, {'val': 6.0}, {'val': 2.0}, {'location': [4, 6], 'val': 25.0}, {'val': 5.0}, {'val': 2.0}]
set of values= set([2.0, 36.0, 5.0, 6.0, 11.0, 25.0])
6*2-1 26**$n2-25**$n2 [[u'-', 11.0, [0, 4]], [[u'*', 12.0, [0, 2]], [6.0], [2.0]], [1.0]] {u'$n2': 6.0}
flattened= [{'location': [0, 4], 'val': 11.0}, {'location': [0, 2], 'val': 12.0}, {'val': 6.0}, {'val': 2.0}, {'val': 1.0}]
set of values= set([1.0, 2.0, 11.0, 12.0, 6.0])
6^2-5^2 26**$n2-25**$n2 [[u'-', 11.0, [0, 6]], [[u'^', 36.0, [0, 2]], [6.0], [2.0]], [[u'^', 25.0, [4, 6]], [5.0], [2.0]]] {u'$n2': 3.0}
flattened= [{'location': [0, 6], 'val': 11.0}, {'location': [0, 2], 'val': 36.0}, {'val': 6.0}, {'val': 2.0}, {'location': [4, 6], 'val': 25.0}, {'val': 5.0}, {'val': 2.0}]
set of values= set([2.0, 3

In [9]:
type((1,2,3))==tuple

True

In [35]:
A=[1,2,3]
A.append(4)
A

[1, 2, 3, 4]

In [39]:
X={'a':'a'}
X.has_key('a')

True